In [ ]:
%load_ext autoreload
%autoreload 2
from __future__ import print_function, division

import numpy as np
#import pandas as pd

import astropy.units as u
import astropy.coordinates as apycoord
from astropy.nddata import block_replicate
from astropy.table import Table, Column, MaskedColumn
from astropy.wcs import WCS
from astropy.io import fits
from astropy.time import Time

#from astropy.stats import sigma_clip
from astropy.modeling import models, fitting
import scipy.stats as stats

from glowing_waffles.differential_photometry import catalog_search, in_frame
from glowing_waffles.differential_photometry  import filter_transform
from glowing_waffles.io import parse_aij_table
from notebook_functions import scale_and_downsample, source_ra, source_error, source_column, uniformize_source_names, \
    find_apass_stars, find_known_variables, plot_apass_variables, get_RA_Dec, get_color, plot_magnitudes, \
    color_corrections, mag_error, corrected_curveses
#import notebook_functions
from ccdproc import CCDData, ImageFileCollection

from gatspy.periodic import LombScargleFast

from scipy import optimize

from glob import glob

%matplotlib inline
import matplotlib.pyplot as plt


# Read image

In [ ]:
ccd = CCDData.read('Tres-3b-004R.fit', unit='adu')   #Read in a single fits /image

In [ ]:
apass, apass_x, apass_y, apass_in_bright, in_apass_x, in_apass_y = find_apass_stars(ccd)
vsx, vsx_x, vsx_y, vsx_names = find_known_variables(ccd)

In [ ]:
disp = scale_and_downsample(ccd.data, downsample=1)

In [ ]:
plot_apass_variables(ccd, disp, vsx_x, vsx_y, vsx_names, apass, in_apass_x, in_apass_y, apass_x, apass_y)

In [ ]:
#Get the magnitudes of the apass stars
apass_mags = apass_in_bright['r_mag']
apass_ra, apass_dec = get_RA_Dec(apass_in_bright)
apass_color, apass_color_error = get_color(apass_in_bright)


all_apass_ra, all_apass_dec = get_RA_Dec(apass)
all_apass_color, all_apass_color_error = get_color(apass)



#Read in the raw measurements file in case the parse function doesn't work as expected
aij_raw = Table.read('tres_3_measurements_4.csv')
#Get the sources
sources = uniformize_source_names(aij_raw)
 
#Use glowing waffles to parse the measurements file
aij_stars = parse_aij_table('tres_3_measurements_4.csv')

#get the ra (which aij gives in hour angle) from the raw aij data
aij_ra = [np.mean(aij_raw[source_ra(source)])*u.hourangle for source in sources]
#get the dec from the raw aij data
aij_dec = [star.dec.mean() for star in aij_stars]
#get the julian date from parsed measurements file
aij_jd = aij_stars[0].jd_utc_start

#Calculate the aij instrumental magnitudes fromt he raw measuremnets file to compare to those from glowing waffles
aij_mags = [-2.5*np.log10(aij_raw[source_column(source)])+2.5*np.log10(aij_raw['EXPOSURE']) for source in sources]

#turn that magnitude list into an array
aij_mags = np.array(aij_mags)

#Create a list of aij coordinates using apycoord.SkyCoord function
aij_coordinates = apycoord.SkyCoord(aij_ra, aij_dec, unit=(u.deg, u.deg))
#create a list of well defined apass stars' coordinates using apycoord.SkyCoord function
apass_coordinates = apycoord.SkyCoord(apass_ra, apass_dec, unit='deg')
#create a list of all of the apass stars' coordiantes using apycoord.SkyCoord function
apass_coordinates_for_color = apycoord.SkyCoord(all_apass_ra, all_apass_dec, unit='deg')

# Are the stars in the image?

In [ ]:
#Get the wcs information of an image to determine whether star is in the image
wcsdict = {}
#/Users/erinaadland/documents/College Work/TrES-3
directories = glob('/Volumes/Erin/20??-??-??/Reduced')
for directory in directories:
    ic = ImageFileCollection(directory)
    for header, filename in ic.headers(object = 'tres-3b', return_fname = True):
        wcs = WCS(header)
        a_time = Time(wcs.wcs.dateobs, out_subfmt='date', scale='utc')
        datename = str(a_time) + '/' + str(filename)
        wcsdict[datename] = wcs

In [ ]:
plt.plot(aij_mags[4])     #Plot one of the stars just to see what it looks like
plt.ylabel('Instrumental Mag')
plt.xlabel('Image')

#compare different ways to get magnitudes.
print(aij_mags[0][0], aij_stars[0].magnitude[0])
print(-2.5*np.log10(174940.8)+2.5*np.log10(60))

In [ ]:
#find the matches of the apass stars and the aij stars. apass_index is a list where each index cooresponds to the index of
#the aij stars and the value of that index corresponds to the index of the matchin apass star
apass_index, d2d, d3d = apycoord.match_coordinates_sky(aij_coordinates, apass_coordinates)

#create a boolean of all of the matches that have a discrepancy of less then 5 arcseconds
good_match = d2d < 5*u.arcsecond

#same matching thing but for ALL of the apass stars
apass_index_for_color, d2d, d3d = apycoord.match_coordinates_sky(aij_coordinates, apass_coordinates_for_color)
good_match_for_color = d2d < 5*u.arcsecond
#Issue 7/22/2016 the number of matches are very few but the image
#overlay in the cells above shows many pretty good looking matches.
#is there a problem with the coordinate comparisons? - SOLVED

# APASS Filter Corrections
## Transform the APASS r magnitudes into R magnitudes using APASS r and i magnitudes

The equation used is R-feder - r-apass = A*c**3 + B*c**2 + C*c + D

Where...

In [ ]:
#Transform the apass magnitudes into the R filter we use
apass_R_mags = filter_transform(apass_in_bright, 'R', g='g_mag', r='r_mag', i='i_mag', transform='ivezic')

In [ ]:
# old_apass_R_mags = apass_R_mags

## Color Corrections Using astropy fit

In [ ]:
%%time
corrections, BminusV, Rminusr = color_corrections(aij_stars, aij_mags,apass_index, apass_color, apass_R_mags, good_match)

In [ ]:
foo2 = np.array(corrections)

In [ ]:
plt.figure(figsize=(15, 5))
#This just plots the slope of the linear fit for all images
plt.plot(foo2[:, 0], '.')
plt.xlabel('Image number')
plt.ylabel('$\\alpha$ (color term)')
plt.grid()
#for image in range(aij_mags.shape[1]):
    #plt.xlim(0,100)
    #plt.scatter(image, corrections[image][0])

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(foo2[:, 1], '.')
plt.xlabel('Image number')
plt.ylabel('$\\gamma$ (zero point)')
plt.grid()

• Source Radius: radius of the aperture used to calculate
net integrated counts. In fixed aperture mode, this value
is the aperture radius set by the user. In variable aperture
mode, this value represents the actual aperture radius
calculated as the product of the average FWHM in
the image and the multiplicative factor set in the MultiAperture
Measurements set up panel.

• FWHM mult: in variable aperture mode, this value is
the FWHM multiplier set in the Multi-Aperture Measurements
set up panel. In fixed aperture mode, this
column is not included in the table.

• Source Rad(base): in variable aperture mode, this
value represents the fixed aperture radius set by the user
and should be set to a number greater than 1.5 times the
maximum FWHM expected to ensure proper measurement
of FWHM. In fixed aperture mode, this column is
not included in the table.

• Sky Rad(min): radius of the inner edge of the annulus
used to calculate the sky background

• Sky Rad(max): radius of the outer edge of the annulus
used to calculate the sky background

In [ ]:
#Calculate the error in the magnitudes to be used in the apass calibration
#define the gain of the ccd
gain = 1.5
#define the read noise of the ccd
read_noise = 30.0


npix = np.pi * 15**2  # pixel^2, estimated aperture size
n_sky = 50.0   # counts/pixel, estimated upper limit

mag_err = mag_error(aij_raw, gain, read_noise, sources)

In [ ]:
%%time
corrected_curves = corrected_curveses(aij_mags, aij_stars, all_apass_color, all_apass_color_error, apass_index_for_color, BminusV, corrections)

#Plot the corrected light curve of star index 0
plt.plot(corrected_curves[5])    #looks kinda screwed up... shouldn't they meet into eachother smoothly with the 
                                 #apass corrections. They are on different nights and all but still...?

In [ ]:
# old_corrected_curves = corrected_curves.copy()
corrected_curves.shape

## Next cell Loops over corrected curves and throws out stars not in frame

In [ ]:
ct = 0
for row in aij_raw:
    r_time = Time(row['JD_UTC'], format = 'jd', out_subfmt='date', scale='utc')
    r_time.format = 'iso'
    filename = row['Label']
    datename = str(r_time) + '/' + str(filename)
    ins = in_frame(wcsdict[datename], aij_coordinates, padding = 100)
    corrected_curves[~ins, ct] = np.nan
    #for star_num in range(0,111):
    #    if in_frame(wcsdict[datename], aij_coordinates[star_num], padding = 100) == False: 
    #        corrected_curves[star_num][ct] = np.nan
    ct += 1
   #  print(ct)

In [ ]:
#create an array of the dates each image was taken at (JD rounded to day)
night = np.array(np.floor(np.array(aij_stars[0].mjd_start + 0.5)) -1)
#find the unique nights in the array of nights
unique_nights = np.unique(night)
#take out one of the nights if you want
unique_nights = set(unique_nights)# - set([57249.0])
#sort the list of nights obviously
unique_nights = sorted(unique_nights)
#find the number of nights to be used for plotting
number_of_nights = len(unique_nights)

night_stdev = [[] for night_n in range(number_of_nights)]
for index, star in enumerate(aij_stars):
    
    #loop over all of the nights and their index in unique nights
    for i, this_night in enumerate(unique_nights):
        #create a night mask that is a boolean in the shape of night
        night_mask = (night == this_night)
        stdev = np.std(corrected_curves[index][night_mask])
        night_stdev[i].append(stdev)

comp_stars = []

for index, stdevs in enumerate(night_stdev):
    good_comps = list(np.argpartition(np.array(stdevs),5)[2:7])
    comp_stars.append(good_comps)

## Functions for making single magnitude plot
### This section is taken from the multi-night photometry notebook from a kelt-1 notebook.

In [ ]:
#define array for the comparison magnitudes
corrected_counts = 10**(-(corrected_curves-2.5*np.log10(aij_raw['EXPOSURE']))/2.5)
comp_counts = []

for i, this_night in enumerate(unique_nights):
        #create a night mask that is a boolean in the shape of night
        night_mask = (night == this_night)
        comp = np.zeros(sum(night_mask))
        for star in comp_stars[i]:
            if star not in (4, 12, 17):
                comp += corrected_counts[star][night_mask]
        comp_counts += list(comp)
comp_counts = np.array(comp_counts)

#calculate the differential magnitudes by simply taking the difference between the corrected curves and the comparison magnitudes"""
diff_corrected = -2.5*np.log10(corrected_counts/comp_counts)

In [ ]:
plt.subplot(2,2,1)
#Plot the instrumental magnitudes
plt.plot(aij_mags[10][0:143])
plt.title('Instrumental')

plt.subplot(2,2,2)
#plot the apass corrected curves
plt.plot(corrected_curves[10][0:143])
plt.title('Corrected')

plt.subplot(2,2,3)
#plot the comparison stars curve
plt.plot(-2.5*np.log10(comp_counts[0:143]))
plt.title('comparison')

plt.subplot(2,2,4)
#plot the difference between the apass star and the comparison star.
plt.plot(diff_corrected[10][0:143])
plt.title('Differential')
plt.show()
for count in range(30):
    plt.plot(diff_corrected[count])
    plt.show()

In [ ]:
corrected_curves

In [ ]:
#print the first point in the first star of the corrected curves as a check for an external check that was done
print(corrected_curves[0][0])

In [ ]:
# loop over all of the stars and their index
max_powers = []
for index, star in enumerate(aij_stars):
    if (index + 1) == 35:
        max_powers.append(np.nan)
        continue
    #start a figure...
    plt.figure(figsize=(5*number_of_nights, 5))
    
    #define a list for night means
    night_means = []
    #define a list for night standard deviations
    night_stds = []
    #define a list for night bins? I suppose
    night_bins = []
    #get the color of the star
    BminusV = all_apass_color[apass_index_for_color[index]]
    #loop over all of the nights and their index in unique nights
    for i, this_night in enumerate(unique_nights):
        plt.subplot(1, number_of_nights + 4, i + 1)
        #create a night mask that is a boolean in the shape of night
        night_mask = (night == this_night)
        #get the night mean and std from the plot magnitudes function and plot the magnitudes
        night_mean, night_std = plot_magnitudes(mags=corrected_curves[index, night_mask], 
                                                times=aij_raw['BJD_TDB'][night_mask], source = index+1,
                                                night = this_night, color = BminusV)
        
        night_means.append(night_mean)
        night_stds.append(night_std)
        night_bins.append(this_night)
        
    """, errors = corrected_curves_er[index][night_mask]"""

    ################# EVERYTHING BELOW THIS POINT PLOTS THE LAST PLOT ################################################
    plt.subplot(1, number_of_nights + 4, number_of_nights + 1)
    # Plot variation over nights.
    night_means = np.array(night_means)
    plt.errorbar(night_bins, night_means, yerr=night_stds, fmt='o')
    variation = night_means.max() - night_means.min()
    size = 10000*variation
    y_lims = plt.ylim()
    y_range = plt.ylim()[1] - plt.ylim()[0]
    plt.scatter([0.8*(plt.xlim()[1]-plt.xlim()[0]) + plt.xlim()[0]], 
                [0.8*(y_range) + plt.ylim()[0]], 
                c='green', marker='o', s=size)
    
    # Plot bar proportional to Lomb-Scargle power.
    bad_mags = np.isnan(corrected_curves[index, :]) | np.isinf(corrected_curves[index, :])
    #bad_errs = np.isnan(mag_err[index])
    bads = bad_mags #| bad_errs
    good_mags = ~bads
    model = LombScargleFast().fit(aij_raw['BJD_TDB'][good_mags], 
                                  corrected_curves[index, good_mags])
    #model = LombScargle(Nterms=1)
    #model.fit(aij_raw['BJD_TDB'][good_mags], 
    #                              corrected_curves[index][good_mags])
    periods, power = model.periodogram_auto(nyquist_factor=100, oversampling=20)
    max_pow = power.max()
    max_powers.append(max_pow)
    print(index + 1, max_pow)
    if max_pow > 0.5:
        color = 'green'
    elif max_pow > 0.4:
        color = 'cyan'
    else:
        color = 'gray'
    
    bar_x = (night_bins[-2] + night_bins[-1])/2
    
    plt.plot([bar_x, bar_x], [plt.ylim()[0], max_pow * y_range + plt.ylim()[0]], 
             color=color, linewidth=10)
    plt.ylim(*y_lims)
    
    ################################ Phase Plots ###################################################
    # ref_time is the Julian date of Jan. 1st, 2011
    ref_time = 2455562.5
    times = aij_raw['BJD_TDB']
    mags = corrected_curves[index, :]

    phase_times = []
    
    model.optimizer.period_range = (0.01, 10.0)
    
    #Phase plots using Find_Best_Periods
    periods_best = model.find_best_periods(n_periods = 2, return_scores = False)
    
    #print(model.best_period)
    #print(model.best_period*2)
    print(periods_best)
    phase_times = ((times - ref_time) % periods_best[0]) / periods_best[0]
    #for i_time in times:
    #    #new_time = (i_time - ref_time)%model.best_period
    ##    new_time = (i_time - ref_time)%periods_best[0]
    #    period_time = new_time/periods_best[0]
    #    phase_times.append(period_time)
        
        
        
    plt.subplot(1,number_of_nights + 4, number_of_nights + 2)
    plt.scatter(phase_times, mags)
    plt.title('Best')
    
    '''
    #Plot Twice the best period
    phase_times_2 =[]
    for i_time in times:
        #new_time_2 = ((i_time - ref_time) / (2 * model.best_period)) - ((i_time - ref_time) // (2 * model.best_period))
        #period_time_2 = new_time_2 / (2 * model.best_period)
        #phase_times_2.append(new_time_2)
        new_time_2 = (i_time - ref_time)%periods_best[1]
        period_time_2 = new_time_2/periods_best[1]
        phase_times_2.append(period_time_2)
    
    #Plot Lomb-Scargle plot
    
    '''
    plt.subplot(1,number_of_nights + 4, number_of_nights + 3)
    phase_times = ((times - ref_time) % (2 * periods_best[0])) / (2 *periods_best[0])
    plt.scatter(phase_times, mags)
    plt.title('2x Best')
    phase_times = ((times - ref_time) % periods_best[1]) / periods_best[1]
    plt.subplot(1,number_of_nights + 4, number_of_nights + 4)
    plt.scatter(phase_times, mags)
    plt.title('Second best')

    """
    #plt.scatter(phase_times_2, mags)
    ct = 0
    for i_pow in power:
        if i_pow == max_pow:
            print(periods[ct])
        ct += 1
    print(periods_best[0])
    plt.scatter(periods,power)
    plt.xlim(0,10)
    """
    plt.show()

In [ ]:
phase_times = ((times - ref_time) % 0.04375)/0.04375
plt.plot(phase_times, corrected_curves[91, :], '.')
plt.ylim(14.8, 14.4)

In [ ]:
### from astropy.stats import LombScargle

In [ ]:
bad_mags = np.isnan(corrected_curves[9]) | np.isinf(corrected_curves[9])
    #bad_errs = np.isnan(mag_err[index])
bads = bad_mags #| bad_errs
good_mags = ~bads
foo = LombScargle(aij_raw['BJD_TDB'][good_mags] * u.day, corrected_curves[9][good_mags])

In [ ]:
f, p = foo.autopower(nyquist_factor=100, samples_per_peak=20, method='fast')

In [ ]:
1/f.min(), 1/f.max()

In [ ]:
f.unit

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(1/f, p)
plt.xlim(0, 0.35)

In [ ]:
curves = Column(data=corrected_curves, name='mag')
errs = [np.array(aij_stars[i].magnitude_error) for i in range(len(aij_stars))]
error_colum = Column(data=errs, name='mag_err')
powers = Column(data=max_powers, name='LS_power')
ids = [np.array(aij_stars[i].id) for i in range(len(aij_stars))]
id_col = Column(data=ids, name='ID')
coord_col = Column(data=aij_coordinates, name='coords')
ra_col = Column(data=aij_coordinates.ra.degree, name='RA')
dec_col = Column(data=aij_coordinates.dec.degree, name='Dec')
apass_name = [apass['recno'][idx] if match else '' for idx, match in zip(apass_index_for_color, good_match_for_color) ]
apass_col = Column(data=apass_name, name='APASS ID')

In [ ]:
apass_col[~good_match_for_color]

In [ ]:
vsx

In [ ]:

vsx_c = apycoord.SkyCoord(ra=vsx['RAJ2000'], dec=vsx['DEJ2000'])
all_vsx_index, d2d, d3d = apycoord.match_coordinates_sky(aij_coordinates, vsx_c)

good = d2d <= 2*u.arcsec

In [ ]:
good.sum()

In [ ]:
vsx_names = [vsx['Name'][i] if match else '' for i, match in zip(all_vsx_index, good)]
vsx_names = Column(data=vsx_names, name='VSX ID')

vsx_period = [vsx['Period'][i] if match else 0 for i, match in zip(all_vsx_index, good)]
vsx_period = Column(data=vsx_period, name='VSX Period')

In [ ]:
calib_table = Table([id_col, ra_col, dec_col, curves, error_colum, powers, apass_col, vsx_names, vsx_period])

In [ ]:
calib_table.write('tres_meas_4_table.fits')

In [ ]:
aijs = aij_stars[0]

In [ ]:
bjd_col = Column(data=aijs.bjd_tdb, name='BJD')
tbjd = Table([bjd_col])
tbjd.write('bjd.fits')

In [ ]:
aij_raw['BJD_TDB']

In [ ]:
#Plot Lomb Scargle
for index, star in enumerate(aij_stars):
    if (index + 1) == 35:
        continue

    # Plot bar proportional to Lomb-Scargle power.
    bad_mags = np.isnan(corrected_curves[index]) | np.isinf(corrected_curves[index])
    #bad_errs = np.isnan(mag_err[index])
    bads = bad_mags #| bad_errs
    good_mags = ~bads
    model = LombScargleFast().fit(aij_raw['BJD_TDB'][good_mags], 
                                  corrected_curves[index][good_mags])
    periods, power = model.periodogram_auto(nyquist_factor=100, oversampling=20)
    max_pow = power.max()
    
    model.optimizer.period_range = (0.01, 5.0)
    #Phase plots using Find_Best_Periods
    periods_best = model.find_best_periods(n_periods = 2, return_scores = False)
    
    print(index + 1)
    ct = 0
    for i_pow in power:
        if i_pow == max_pow:
            print(periods[ct])
        ct += 1
    print(periods_best[0])
    plt.plot(periods,power)
    plt.xlim(0,5)
    plt.show()

In [ ]:
from astropy.table import Table, Column
t = Table(rows = corrected_curves)
cols = [Column(curve, name=str(idx+1)) for idx, curve in enumerate(corrected_curves)]
t_with_date = Table(cols)
date = aij_raw['BJD_TDB']
t_with_date.add_column(date)
t_with_date.write('updated_measurements_4.csv', format = 'csv')
print(t_with_date)
#print(comp_stars[0])

In [ ]:
%matplotlib inline
#loop over all of the stars and their index
for index, star in enumerate(aij_stars):
    #start a figure...
    plt.figure(figsize=(5*number_of_nights, 5))
    
    #define a list for night means
    night_means = []
    #define a list for night standard deviations
    night_stds = []
    #define a list for night bins? I suppose
    night_bins = []
    #get the color of the star
    BminusV = all_apass_color[apass_index_for_color[index]]
    #loop over all of the nights and their index in unique nights
    for i, this_night in enumerate(unique_nights):
        plt.subplot(1, number_of_nights + 1, i + 1)
        #create a night mask that is a boolean in the shape of night
        night_mask = (night == this_night)
        #get the night mean and std from the plot magnitudes function and plot the magnitudes
        night_mean, night_std = plot_magnitudes(mags=diff_corrected[index][night_mask], 
                                                times=aij_raw['BJD_TDB'][night_mask], source = index+1,
                                               night = this_night, color = BminusV)
""", errors = corrected_curves_er[index][night_mask]"""

In [ ]:
apass_r_mags = apass_in_bright['r_mag']
apass_R_mags = filter_transform(apass_in_bright, 'R', r='r_mag', i='i_mag')
apass_bv = apass_in_bright['B-V']
aij_R = 
apassr_i = apass_in_bright['r_mag'] - apass_in_bright['i_mag']
apassR_r = apass_R_mags - apass_r_mags 

plt.plot(apass_bv, apassR_r, 'o')
#plt.plot(apassr_i, apassR_r, '-')

In [ ]:
A = -0.0107
B = 0.0050
C = -0.2689
D = -0.1540
c = np.arange(-0.30,-0.16,50)
R_mag = (A * (c**3)) + (B * (c**2)) + (C * c) + D + apass_r_mags
plt.plot()

In [ ]:
from scipy import optimize
#Huber Loss Function test from 'Statistics, Data Mining, and Machine Learning in Astronomy' (Ivezic et al. 2014)
y = np.array(testRminusr)
x = np.array(testBminusV)
dy = np.zeros(14)+0.01
print(len(x), len(y), len(dy))

# Define the standard squared-loss function
def squared_loss(m, b, x, y, dy):
    y_fit = m * x + b
    return np.sum(((y - y_fit) / dy) ** 2, -1)


# Define the log-likelihood via the Huber loss function
def huber_loss(m, b, x, y, dy, c=2):
    y_fit = m * x + b
    t = abs((y - y_fit) / dy)
    flag = t > c
    return np.sum((~flag) * (0.5 * t ** 2) - (flag) * c * (0.5 * c - t), -1)

f_squared = lambda beta: squared_loss(beta[0], beta[1], x=x, y=y, dy=dy)
f_huber = lambda beta: huber_loss(beta[0], beta[1], x=x, y=y, dy=dy, c=1)

#------------------------------------------------------------
# compute the maximum likelihood using the huber loss
beta0 = (2, 30)
beta_squared = optimize.fmin(f_squared, beta0)
beta_huber = optimize.fmin(f_huber, beta0)

print(beta_squared)
print(beta_huber)
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111)

x_fit = np.linspace(0, 20, 10)
ax.plot(x_fit, beta_squared[0] * x_fit + beta_squared[1], '--k',
        label="squared loss:\n $y=%.2fx + %.1f$" % tuple(beta_squared))
ax.plot(x_fit, beta_huber[0] * x_fit + beta_huber[1], '-k',
        label="Huber loss:\n $y=%.2fx + %.1f$" % tuple(beta_huber))
ax.legend(loc=4)

ax.errorbar(x, y, dy, fmt='.k', lw=1, ecolor='gray')

#ax.set_xlim(0, 350)
#ax.set_ylim(100, 700)
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')

plt.show()

In [ ]:
arf = aij_stars[0]

In [ ]:
arf._table.colnames